In [1]:
gsea.dir <- "/data2/mito_lineage/software/Bioinformatics_Tools/"
DE.out.path <- "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters"
export.path <- "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/GSEA/"


In [2]:
dir.create(DE.out.path)

Warning message in dir.create(DE.out.path):
“'/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters' already exists”


In [3]:
source(paste0(gsea.dir,'R/Gsea.R'))

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with


In [4]:
cols.prior <- c("Gene","baseMean","log2FoldChange","lfcSE","stat","pvalue","padj")
cols.needed <- c("Gene","log2FoldChange")
cols.curr <- c("X","p_val","avg_log2FC","pct.1","pct.2","p_val_adj") #first col is empty in csv

In [5]:
#for f in list.files(DE.path, pattern="*.csv"){

DE_files <- list.files(path = DE.out.path,pattern="*.csv", full.names=TRUE)
for(f in DE_files){
    print(f)
    curr <- read.csv(f)
    if (!(nrow(curr)==0)){
        colnames(curr) <- c("Gene","pvalue","log2FoldChange","pct.1","pct.2","padj")
        write.table(curr, file.path(DE.out.path, paste0(strsplit(basename(f), ".csv")[[1]], ".tsv")),row.names=FALSE, sep='\t')
    }
    
}

[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_0.DE.GeneActivity.csv"
[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_1.DE.GeneActivity.csv"
[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_10.DE.GeneActivity.csv"
[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_11.DE.GeneActivity.csv"
[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_12.DE.GeneActivity.csv"
[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_13.DE.GeneActivity.csv"
[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_14.DE.GeneActivity.csv"
[1] "/data2/isshamie/m

In [6]:
#rm(list=ls())

########################################################################################
##  gsea_ex1
#---------------------------------------------------------------------------------------
#' @description This example is designed to demonstrate how to run gsea analysis on the 
#'              input files (DESeq2 results) put in the same directory
#' @details All the parameter definitions is in the below section of "Setting Parameter"
#' @return out a list that summarizes the GSEA enrichment analysis
#' 
#' @export exportExcel_fp an excel file of the 'out' can be exported
#' @export exportPDF_fp an figure of the GSEA enrichment analysis
#' --------------------------------------------------------------------------------------------------
#' @author Austin W.T. Chiang
#' @note   v1.4 -- 2017.02.15 Add description of ex1, ex2, and idmapping file
#' @note   v1.3 -- 2017.02.08 Add function of displaying topN pathways and optimze code
#' @note   v1.2 -- 2017.02.06 Add zebrafish2 human id mapping
#' @note   v1.1 -- 2017.01.19 Midified for BEN
#' @note   v1.0 -- 2016.12.15 File created
#' --------------------------------------------------------------------------------------------------
########################################################################################
gsea_ex1 <- function(DE_out_path = NULL, export_path = NULL,
                     IDmappingfile = NULL, convert = FALSE,
                     gene_name = TRUE, GSEA_set = c('c0', 'c2', 'c3'),
                     minGSSize = 30, pAdjustMethod = "none",
                     pvalueCutoff = 0.01, export = "BOTH",
                     ShowTop = TRUE, TopN = 5){
  DE_out_files <- list.files(path = DE_out_path,pattern="*.tsv", full.names = TRUE)
  for(f in DE_out_files){
    print(paste0("Here we go .... :", f))
    #f <- "res_geno_dobs_removed_C57BL6_J__M.tsv"
    ## Input the DESeq result file
    myClass <- read.csv(f, row.names = 1,sep="\t")
    if(convert == TRUE){
      geneMapTable <- read.csv(IDmappingfile, sep = "\t")
      myClass <- useIDmappingFile(myClass=myClass,
                                  geneMapTable=geneMapTable) #,
                                  #Original_attribute=Original_attribute,
                                 # Convert_attribute=Co nvert_attribute)
    }
    
    #DEGTable <- subset(myClass, abs(log2FoldChange) >= .58 & (padj < 0.05) )
    DEGTable <- subset(myClass, (padj < 0.1) )
    DEGTable <- DEGTable[with(DEGTable, order(-log2FoldChange)), ]
    print(colnames(DEGTable))
    if(gene_name==TRUE){
      DEGTable$Gene <- DEGTable$gene_name
    }else{
      print('here')
      DEGTable$Gene <- row.names(DEGTable)
    }

    DEGTable <- DEGTable[,c("Gene","log2FoldChange")]
    
    ## do the GSEA analysis
    f1 <- gsub(".csv", "", basename(f))
    exportExcel_fp = paste0(export_path,f1,"_gsea.xlsx")
    exportPDF_fp = paste0(export_path,f1,"_gsea.pdf")
    GSEA_res <-  AnnotDEG_byGSEA(DEGTable = DEGTable, GSEA_set = GSEA_set,
                                 export = export, ShowTop = ShowTop, TopN = TopN,
                                 exportExcel_fp = exportExcel_fp,
                                 exportPDF_fp = exportPDF_fp,
                                 minGSSize = minGSSize, 
                                 pAdjustMethod = pAdjustMethod, 
                                 pvalueCutoff = pvalueCutoff)
    
  }
}


########################################################################################
##  Setting Parameter
########################################################################################
#' @param DE_out_path the path of the DESeq2 output files 
#' @param export_path the path that user want to export the GSEA results
#' @param convert 'TRUE': need to do id converion; 'FALSE': no id conversion needed  
#' @param IDmappingfile the idmapping file show that user's id to human gene name
#' @param gene_name 'TRUE': there is a columan "gene_name" in the input files (i.e. DESeq2 output files)
#' @param GSEA_set = c('c0', 'c2', 'c3')
#'                 co - hallmark gene sets;       c1 - positional gene sets; 
#'                 c2 - curated gene sets;        c3 - motif gene sets;
#'                 c4 - computational gene sets;  c5 - GO gene sets;
#'                 c6 - oncogenic signatures;     c7 - immunologic signatures. 
#'                 (http://software.broadinstitute.org/gsea/msigdb/collections.jsp#C2)
#' @param minGSSize minimum size of genes for a specific GSEA gene set
#' @param pAdjustMethod one of "holm", "hochberg", "hommel", "bonferroni", "BH", "BY", "fdr", "none"
#' @param pvalueCutoff cutoff of pvalue in the GSEA enrichment analysis 
#' @param export option for exporting GSEA result: "EXCEL", "PDF", or "BOTH"
#' @param ShowTop 'TRUE': the visualization will display only the top and bottom N gene sets; 
#'                'FALSE': all of the significant gsea gene sets will be displayed 
#'                        --> Note that: it is usually results in too many gene sets, which are not easy to see anything    
#' @param TopN this parameter indicate how many gene sets need to be displayed in the plots
########################################################################################

# DE_out_path <- DE.out.path
# export_path <- 
#IDmappingfile <- "" #"mouse2human.txt" #"Data/zebrafish2human_idmapping.txt"
convert = FALSE
gene_name = FALSE 
GSEA_set = c('c0', 'c2', 'c3', 'c7')
minGSSize = 10 #20 #30
pAdjustMethod = "none"
pvalueCutoff = 0.1
export = "BOTH"
ShowTop = TRUE 
TopN = 10


########################################################################################
##  RUN gsea_ex1
########################################################################################
gsea_ex1(DE_out_path = DE.out.path, export_path = export.path, gene_name = gene_name,
         convert = convert, GSEA_set = GSEA_set, minGSSize = minGSSize,
         pAdjustMethod = pAdjustMethod,  pvalueCutoff = pvalueCutoff,
         export = export, ShowTop = ShowTop,   
         TopN = TopN)

[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_0.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"
[1] "small"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_1.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"
[1] "small"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_10.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"


preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.07% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...

preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.07% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...

preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.07% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgseaMultilevel(...):
“For some pathwa

[1] "preExcel writing"
[1] "postExcel writing"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_11.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"


preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):

[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_12.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"
[1] "small"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_13.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"


...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...

preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...

preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...

Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to 

[1] "preExcel writing"
[1] "postExcel writing"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_4.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"
[1] "small"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_5.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"
[1] "small"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_6.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"
[1] "small"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_7.DE

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...

preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...

preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...

Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to 

[1] "preExcel writing"
[1] "postExcel writing"
[1] "Here we go .... :/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/cluster_9.DE.GeneActivity.tsv"
[1] "pvalue"         "log2FoldChange" "pct.1"          "pct.2"         
[5] "padj"          
[1] "here"


preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

preparing geneSet collections...

GSEA analysis...

no term enriched under specific pvalueCutoff...

Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):
“summary method to convert the object to data.frame is deprecated, please use as.data.frame instead.”
Warning message in is.na(egmt[[c]]):
“is.na() applied to non-(list or vector) of type 'S4'”
Warning message in summary(egmt[[c]]):

In [7]:
export.path

[1] "/data2/isshamie/mito_lineage/output/annotation/run_vDataBeforeCorrupt/TcellDupi_may17_2021/DE/clusters/GSEA/"